In [ ]:
%matplotlib inline

In [ ]:
import os
import sys
schrodinger_folder = os.path.join('..', 'schrodinger') # path to the schrodinger package downloaded from https://github.com/agolovanov/schrodinger
sys.path.append(schrodinger_folder)
import decay # decay.py which comes with the supplementary materials
import wavefunction
import potential
import solver
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import ImageGrid
import numpy as np
import eigen
from scipy import optimize, integrate, interpolate

In [ ]:
figure_width = 3.37
figure_height = 0.7 * figure_width
plt.rc('figure', figsize=(figure_width, figure_height), dpi=200)
plt.rc('font', size=10)
plt.rc('text', usetex=True)
plt.rc('lines', linewidth=1.4)
plt.rc('axes', grid=True, labelpad=1.0, axisbelow=True)
plt.rc('grid', linewidth = 0.5, linestyle = '-', color='0.8')
plt.rc('xtick', direction='in')
plt.rc('ytick', direction='in')
plt.rc('xtick.major', size = 0)
plt.rc('ytick.major', size = 0)

# 1D delta potential

In [ ]:
depth = 1.0
fields = [0.3, 1, 5]

v = potential.DeltaPotential1D(depth)
solvers = [solver.CrankNicolsonSolver(100, 0.02, 0.01, potential=potential.UniformField1D(f, potential=v)) for f in fields]
x = solvers[0].x
psi0 = v.get_eigenfunction()(x)

In [ ]:
tmax = 4.1
iters = 50
corrs_numeric = []
corrs_analytic = []
corrs_analytic_bf = []

for f, s in zip(fields, solvers):
    ts, psis = s.execute(tmax, output_dt=tmax/iters, psi0=psi0)
    
    corrs_numeric.append(np.array([np.abs(wavefunction.correlation(x, psi0, psi)) ** 2 for psi in psis]))
    corrs_analytic.append(decay.calculate_state_decay_analytic(x, psi0, ts, f))
    corrs_analytic_bf.append(decay.calculate_state_decay_analytic(x, psi0, ts, f, True))

In [ ]:
def delta_corr(A):
    return ((0.5 * A) ** 2 + 1) ** (-2)

In [ ]:
fig, ax = plt.subplots(nrows=3, figsize=(figure_width, 1.2 * figure_width))

for a, f, c1, c2, c3 in zip(ax, fields, corrs_numeric, corrs_analytic_bf, corrs_analytic):
    a.plot(ts, c1, label='TDSE')
    a.plot(ts, c2, '--', label='Motionless')
    a.plot(ts, c3, ':', label='Free electron')
    #a.plot(ts, delta_corr(f * ts))
    a.set_ylim(0, 1.05)
    a.set_ylabel('$C_0$')
    #a.set_title(f'$E={f:g}$')
    a.set_yticks([0,0.5, 1])
    a.set_xlim(0, 4)
    a.text(0.8, 0.85, f'$E={f:g}$', transform=a.transAxes, bbox={'facecolor':'white', 'pad':1, 'lw':0})
    
ax[0].set_xticklabels([])
ax[1].set_xticklabels([])
ax[2].set_xlabel('$t$')

ax[2].legend(fontsize=9, loc=4)

for a in ax[:2]:
    a.set_yticks([0.5, 1])

plt.tight_layout(pad=0.4, h_pad=0.0)
plt.savefig('fig/1d_delta_corr.eps')

In [ ]:
# for wi(E)

print(1 / (np.sqrt(np.sqrt(np.e) - 1) * 2 * np.sqrt(2)))
print(1 / (np.sqrt(np.sqrt(np.e) - 1) * 2))

alpha1 = 1 / optimize.brentq(lambda A: delta_corr(A) - np.exp(-1), 0, 10)
print(alpha1)

In [ ]:
def model_ionization(alpha):
    return lambda A: np.exp(- alpha * A)

In [ ]:
sq_diff = lambda alpha: integrate.quad(lambda A: (delta_corr(A) - model_ionization(alpha)(A)) ** 2, 0, np.infty)[0]

alpha2 = optimize.minimize(sq_diff, alpha1).x[0]
print(alpha2)

abs_diff = lambda alpha: integrate.quad(lambda A: np.abs(delta_corr(A) - model_ionization(alpha)(A)), 0, np.infty)[0]

alpha3 = optimize.minimize(abs_diff, alpha1).x[0]
print(alpha3)

A = np.linspace(0, 20, 400)
min_diff = lambda alpha: np.max(np.abs(delta_corr(A) - model_ionization(alpha)(A)))

alpha4 = optimize.minimize(min_diff, alpha1).x[0]
print(alpha4)

alpha = np.linspace(0.3, 2, 100)
plt.plot(alpha, np.vectorize(sq_diff)(alpha))
plt.plot(alpha, np.vectorize(abs_diff)(alpha), '--')
plt.plot(alpha, np.vectorize(min_diff)(alpha), ':')

# Soft-core

In [ ]:
fields = [0.05, 0.1, 0.2]

v0 = lambda x: -1 / np.sqrt(x ** 2 + 2)
solvers = {f : solver.SplitOperatorHalfSpectralSolver(100, 0.4, 0.1, potential=(lambda x: -1 / np.sqrt(x ** 2 + 2) - f * x), stationary=True) for f in fields}
x = solvers[fields[0]].x

In [ ]:
w, psis0 = eigen.calculate_eigenstates(x, v0, 10, -0.5)
for i in range(len(psis0)):
    psi = psis0[i]
    if np.max(np.real(psi)) < np.max(-np.real(psi)):
        psis0[i] = -psi
psi0 = psis0[0]

for psi in psis0:
    plt.plot(x, np.real(psi))
    
plt.xlim(-30, 30)

In [ ]:
p, psi_p = wavefunction.momentum_representation(x, psi0)
p0_sq = wavefunction.correlation(p, np.conj(psi_p), p ** 2 * psi_p)
print(p0_sq)
p0 = np.sqrt(np.abs(p0_sq))
print(f'p0 = {p0}')

print(f'p0^3 = {p0 ** 3})')

In [ ]:
n = 4
styles = [[1, 0], [3, 1], [1, 1], [3, 1, 1, 1]]

fig, ax = plt.subplots()

for i in range(n):
    ax.plot(x, np.real(psis0[i]), dashes=styles[i], label=f'$n = {i+1}$')
    
ax.set_xlim(-20, 20)
ax.legend(fontsize='small')
ax.set_xlabel('$x$')
ax.set_ylabel(r'$\psi_n$')
ax.set_ylim(-0.6, 0.6)

plt.tight_layout(pad=0.2)
plt.savefig('fig/softcore_wavefunctions.eps')

In [ ]:
def soft_core_corr(A):
    return np.sum([decay.correlation_p(x, psi, psi0, A) for psi in psis0], axis=0)

In [ ]:
tmax = 30
iters = 50

corrs_numeric = {}
corrs_analytic = {}
corrs_analytic_bf = {}
corrs_numeric_tot = {}
corrs_analytic_tot = {}
corrs_analytic_bf_tot = {}

for f, s in solvers.items():
    ts, psis = s.execute(tmax, output_dt=tmax/iters, psi0=psi0)
    corrs_numeric[f] = [np.array([np.abs(wavefunction.correlation(x, psi0, psi)) ** 2 for psi in psis]) for psi0 in psis0]
    corrs_numeric_tot[f] = np.sum(corrs_numeric[f], axis=0)
    corrs_analytic[f] = [decay.calculate_state_decay_analytic(x, psi0, ts, f, False, psi_comp=psi) for psi in psis0]
    corrs_analytic_tot[f] = np.sum(corrs_analytic[f], axis=0)
    corrs_analytic_bf[f] = [decay.calculate_state_decay_analytic(x, psi0, ts, f, True, psi_comp=psi) for psi in psis0]
    corrs_analytic_bf_tot[f] = np.sum(corrs_analytic_bf[f], axis=0)

In [ ]:
def annotate_axes(ax):
    letters = 'abcde'
    for i, a in enumerate(ax):
        a.text(0.95, 0.88, f'({letters[i]})', ha='center', va='center', fontsize=10, transform=a.transAxes, bbox={'facecolor':'white', 'pad':1, 'lw':0})

In [ ]:
field0 = fields[2]
print(field0)

fig, ax = plt.subplots(nrows = 3, figsize=(figure_width, 1.2 * figure_width))

ns = 3
styles = [[3, 1], [1, 1], [3, 1, 1, 1]]

ax[0].plot(ts, corrs_numeric_tot[field0])
for corr, s in zip(corrs_numeric[field0][:ns], styles):
    ax[0].plot(ts, corr, dashes=s)

ax[1].plot(ts, corrs_analytic_tot[field0], label='$C$')
for i, (corr, s) in enumerate(zip(corrs_analytic[field0][:ns], styles)):
    ax[1].plot(ts, corr, dashes=s, label=r'$C_{%d}$' % i)
    
ax[1].legend(loc=4, fontsize='small')

ax[2].plot(ts, corrs_analytic_bf_tot[field0])
for corr, s in zip(corrs_analytic_bf[field0][:ns], styles):
    ax[2].plot(ts, corr, dashes=s)

for a in ax:
    a.set_xlim(0, 12)
    a.set_ylim(0, 1.05)
    a.set_ylabel('$C$')
    
for a in ax[:2]:
    a.set_yticks([0.5, 1])
    
ax[0].set_xticklabels([])
ax[1].set_xticklabels([])

ax[2].set_xlabel('$t$')
#annotate_axes(ax)

ax[0].text(0.98, 0.8, 'TDSE', ha='right', transform=ax[0].transAxes, bbox={'facecolor':'white', 'pad':1, 'lw':0})
ax[1].text(0.98, 0.8, 'Free electron', ha='right', transform=ax[1].transAxes, bbox={'facecolor':'white', 'pad':1, 'lw':0})
ax[2].text(0.98, 0.8, 'Motionles', ha='right', transform=ax[2].transAxes, bbox={'facecolor':'white', 'pad':1, 'lw':0})

plt.tight_layout(pad=0.05, h_pad=0)

plt.savefig('fig/softcore_multiple.eps')

In [ ]:
fig, ax = plt.subplots(nrows = 3, figsize=(figure_width, 1.2 * figure_width))

for f, a in zip(fields, ax):
    a.plot(ts, corrs_numeric_tot[f], label='TDSE')
    a.plot(ts, corrs_analytic_bf_tot[f], '--', label='Motionless')
    a.plot(ts, corrs_analytic_tot[f], ':', label='Free electron')
    a.set_xlim(0, 20)
    a.set_ylim(0, 1.05)
    #a.set_title(f'$E={f:g}$')
    a.text(0.04, 0.1, f'$E={f:g}$', transform=a.transAxes, bbox={'facecolor':'white', 'pad':1, 'lw':0})
    a.set_ylabel('$C$')
    a.set_yticks([0, 0.5, 1])

for a in ax[:2]:
    a.set_yticks([0.5, 1])

ax[2].legend(fontsize='small', loc=1)

ax[0].set_xticklabels([])
ax[1].set_xticklabels([])

ax[2].set_xlabel('$t$')
plt.tight_layout(pad=0.05, h_pad=0)

plt.savefig('fig/softcore_corr.eps')

In [ ]:
A = np.linspace(0, 5, 100)
soft_core_corr_value = soft_core_corr(A)
soft_core_interp = interpolate.interp1d(A, soft_core_corr_value, fill_value=(1, 0), bounds_error=False)

alpha1 = 1 / optimize.brentq(lambda A: soft_core_interp(A) - np.exp(-1), 0, 10)
print(alpha1)
print(alpha1 / np.sqrt(2))

plt.plot(A, soft_core_corr_value)
plt.plot(A, model_ionization(alpha1)(A))

In [ ]:
sq_diff = lambda alpha: integrate.quad(lambda A: (soft_core_interp(A) - model_ionization(alpha)(A)) ** 2, 0, 10)[0]

alpha2 = optimize.minimize(sq_diff, alpha1).x[0]
print(alpha2)

abs_diff = lambda alpha: integrate.quad(lambda A: np.abs(soft_core_interp(A) - model_ionization(alpha)(A)), 0, 10)[0]

alpha3 = optimize.minimize(abs_diff, alpha1).x[0]
print(alpha3)

A = np.linspace(0, 20, 400)
min_diff = lambda alpha: np.max(np.abs(soft_core_interp(A) - model_ionization(alpha)(A)))

alpha4 = optimize.minimize(min_diff, alpha1).x[0]
print(alpha4)

alpha = np.linspace(0.3, 2, 20)
plt.plot(alpha, np.vectorize(sq_diff)(alpha))
plt.plot(alpha, np.vectorize(abs_diff)(alpha), '--')
plt.plot(alpha, np.vectorize(min_diff)(alpha), ':')

In [ ]:
potential_func = lambda x, E: - 1 / np.sqrt(2 + x ** 2) - E * x
x = np.linspace(0, 12, 5000)
potential_max = lambda E: np.max(potential_func(x, E))

Ecritical = optimize.brentq(lambda E: potential_max(E) + 0.5, 0.06, 0.07)

print(Ecritical)

In [ ]:
def find_closest(tarr, t0):
    i0 = 1
    for i, t in enumerate(tarr):
        if t > t0:
            i0 = i
            break
    return i0 if abs(tarr[i0] - t0) < abs(tarr[i0-1] - t0) else i0-1

In [ ]:
E1 = fields[0]
E2 = fields[2]
print(E1, E2)

ets = [0, 1, 2]

fig = plt.figure(figsize=(figure_width, 1.5 * figure_width))

grid_x = [ImageGrid(fig, 221, nrows_ncols=(3, 1), axes_pad=0, aspect=False), ImageGrid(fig, 223, nrows_ncols=(3, 1), axes_pad=0, aspect=False)]
grid_p = [ImageGrid(fig, 222, nrows_ncols=(3, 1), axes_pad=0, aspect=False), ImageGrid(fig, 224, nrows_ncols=(3, 1), axes_pad=0, aspect=False)]

for f, gx, gp in zip([E1, E2], grid_x, grid_p):
    tmax = 3 / f
    ts, psis = solvers[f].execute(tmax, output_dt=tmax/iters, psi0=psi0)
    x = solvers[f].x
    for i, et in enumerate(ets):
        index = find_closest(ts, et / f)
        psi = psis[index] 
        gx[i].plot(x, np.abs(psi) ** 2, label='TDSE')
        psi_motionless = decay.calculate_psi(x, psi0, ts[index], f, big_field=True)
        gx[i].plot(x, np.abs(psi_motionless) ** 2, '--', label='M')
        psi_free = decay.calculate_psi(x, psi0, ts[index], f, big_field=False)
        gx[i].plot(x, np.abs(psi_free) ** 2, ':', label='FE')
        
        p, psi_p = wavefunction.momentum_representation(x, psi)
        gp[i].plot(p, np.abs(psi_p) ** 2, label='TDSE')
        p, psi_motionless_p = wavefunction.momentum_representation(x, psi_motionless)
        gp[i].plot(p, np.abs(psi_motionless_p) ** 2, '--', label='M')
        p, psi_free_p = wavefunction.momentum_representation(x, psi_free)
        gp[i].plot(p, np.abs(psi_free_p) ** 2, ':', label='FE')

for g in grid_x:
    for a in g:
        a.set_xlim(-6, 6)
        a.set_ylim(0, 0.4)
    g[1].set_ylabel(r'$|\psi|^2$')
        

for g in grid_p:
    for a in g:
        a.set_xlim(-3, 3)
        a.set_ylim(0, 1.4)
    g[1].set_ylabel(r'$|\tilde\psi|^2$')
        
        
for g in grid_x + grid_p:
    for i, a in enumerate(g):
        a.set_yticklabels([])
        a.text(0.05, 0.8, f'$t={ets[i]}/E$', transform=a.transAxes, bbox={'facecolor':'white', 'pad':1, 'lw':0})
        
grid_x[0][-1].set_xticklabels([])
grid_p[0][-1].set_xticklabels([])

grid_x[1][-1].set_xlabel('$x$')
grid_p[1][-1].set_xlabel('$p$')

grid_p[0][0].legend(fontsize='x-small', handlelength=1.5, handletextpad=0.3)

plt.tight_layout(pad=0.1, h_pad=2, w_pad=0.4, rect=[0.03, 0, 1, 0.96])
fig.text(0.54, 0.98, f'$E = {E1}$', ha='center', va='center')
fig.text(0.54, 0.51, f'$E = {E2}$', ha='center', va='center')

plt.savefig('fig/softcore_evolution.eps')

# Coulomb

In [ ]:
x = np.linspace(-20, 20, 101)
r = np.meshgrid(x, x, x, indexing='ij')

v = potential.CoulombPotential()

levels = [(1,0), (2,0), (2,1), (3,0), (3,1), (3,2), (4,0), (4,1), (4,2), (4,3)]

psis = [v.get_eigenfunction(*l, 0)(*r) for l in levels]
psi0 = psis[0]

In [ ]:
def coulomb_corr(a, psi):
    @np.vectorize
    def corrf(a):
        return wavefunction.correlation(r, psi, psi0 * np.exp(-1j * a * r[2]))
    
    return np.abs(corrf(a)) ** 2

@np.vectorize
def coulomb_corr_tot(a):
    return np.sum([coulomb_corr(a, psi) for psi in psis], axis=0)

In [ ]:
a = np.linspace(0, 5, 100)

corrs = [coulomb_corr(a, psi) for psi in psis]
corrs_tot = np.sum(corrs, axis=0)

In [ ]:
E = 0.4
t = a / E
plt.plot(a, corrs_tot, label=r'$\tilde{C}$')

styles = [[3, 1], [1, 1], [3, 1, 1, 1]]
for l, c, s in zip(levels[:3], corrs[:3], styles):
    plt.plot(a, c, dashes=s, label=r'$\tilde{C}_{%d,%d,0}$' % l)
    
plt.xlabel('$A$')
plt.xlim(0, 4)
plt.ylim(0, 1.05)
plt.ylabel(r'$\tilde{C}$')
plt.legend(fontsize=10)
plt.tight_layout(pad=0.2)

plt.savefig('fig/coulomb_corr.eps')

In [ ]:
A = np.linspace(0, 5, 100)
coulomb_corr_value = coulomb_corr_tot(A)
coulomb_core_interp = interpolate.interp1d(A, coulomb_corr_value, fill_value=(1, 0), bounds_error=False)

alpha1 = 1 / optimize.brentq(lambda A: coulomb_corr_tot(A) - np.exp(-1), 0, 3)
print(alpha1)
print(alpha1 / np.sqrt(2))

plt.plot(A, coulomb_corr_value)
plt.plot(A, model_ionization(alpha1)(A))

In [ ]:
sq_diff = lambda alpha: integrate.quad(lambda A: (coulomb_core_interp(A) - model_ionization(alpha)(A)) ** 2, 0, 10)[0]

alpha2 = optimize.minimize(sq_diff, alpha1).x[0]
print(alpha2)

abs_diff = lambda alpha: integrate.quad(lambda A: np.abs(coulomb_core_interp(A) - model_ionization(alpha)(A)), 0, 10)[0]

alpha3 = optimize.minimize(abs_diff, alpha1).x[0]
print(alpha3)

A = np.linspace(0, 20, 400)
min_diff = lambda alpha: np.max(np.abs(coulomb_core_interp(A) - model_ionization(alpha)(A)))

alpha4 = optimize.minimize(min_diff, alpha1).x[0]
print(alpha4)

alpha = np.linspace(0.3, 2, 20)
plt.plot(alpha, np.vectorize(sq_diff)(alpha))
plt.plot(alpha, np.vectorize(abs_diff)(alpha), '--')
plt.plot(alpha, np.vectorize(min_diff)(alpha), ':')

# Tunnel formula

In [ ]:
def w(E):
    return 4 / np.abs(E) * np.exp(- 2 / 3 / np.abs(E))

In [ ]:
atomic_t = 2.418884326505e-17 # s

pulse_length_s = 200e-18

pulse_length = pulse_length_s / atomic_t
print(pulse_length)

amplitude = 10
def E_attosecond(t):
    return amplitude * np.exp(- (t - 3 * pulse_length) ** 2 / pulse_length ** 2)

integral_func_attosecond = np.vectorize(lambda t: integrate.quad(lambda t1: w(E_attosecond(t1)), 0, t)[0])

In [ ]:
lambda_l = 0.9 # um
atomic_t = 2.418884326505e-17 # s
c = 3e8 # m/s
omega_l = 2 * np.pi * c / (lambda_l * 1e-6) * atomic_t
print(omega_l)
T_l = 2 * np.pi / omega_l
print(T_l)

a0 = 10
E_l_SI = 3.21 * a0 * 1e12/ lambda_l
print(f"E = {E_l_SI / 1e12} TV/m")
E_atomic = 5.14220652e11
E_l = E_l_SI / E_atomic
print(E_l)

N_pulse = 60e-15 / atomic_t / T_l
phase = 0

def E_gauss(t):
    return E_l * np.exp(- (t - 1.6 * N_pulse * T_l) ** 2 / (0.5 * N_pulse * T_l) ** 2) * np.cos(omega_l * t + phase)

integral_func_gauss = np.vectorize(lambda t: integrate.quad(lambda t1: w(E_gauss(t1)), 0, t)[0])

In [ ]:
t_attosecond = np.linspace(0, 16, 100)
probability_attosecond = 1 - np.exp(- integral_func_attosecond(t_attosecond))

t_gauss = np.linspace(0, 1000, 400)
probability_gauss = 1 - np.exp(- integral_func_gauss(t_gauss))

In [ ]:
fig, ax = plt.subplots(nrows=2, ncols=2, figsize=(figure_width, figure_width))
ax[0][1].plot(t_attosecond, E_attosecond(t_attosecond))
ax[0][0].plot(t_gauss, E_gauss(t_gauss))

ax[1][1].plot(t_attosecond, probability_attosecond)
ax[1][0].plot(t_gauss, probability_gauss)

ax[0][0].set_ylabel('$E$')
ax[1][0].set_ylabel('$W$')

ax[0][1].set_ylim(-2, 2)
ax[0][0].set_ylim(-0.2, 0.2)

ax[0][0].set_xlim(min(t_gauss), max(t_gauss))
ax[1][0].set_xlim(min(t_gauss), max(t_gauss))

ax[0][1].set_title('200 as pulse')

ax[0][0].set_title('60 fs pulse')

for a in (ax[0][1], ax[1][1]):
    a.set_xlim(min(t_attosecond), max(t_attosecond))
    a.fill_between(t_attosecond, 100 * (np.abs(E_attosecond(t_attosecond)) > 1 / 16), -100 * (np.abs(E_attosecond(t_attosecond)) > 1 / 16), color='#ddddf0', zorder=-20, lw=0)
    
for a in (ax[0][0], ax[1][0]):
    a.set_xlim(min(t_gauss), max(t_gauss))
    a.fill_between(t_gauss, 100 * (np.abs(E_gauss(t_gauss)) > 1 / 16), -100 * (np.abs(E_gauss(t_gauss)) > 1 / 16), color='#ddddf0', zorder=-20, lw=0)

for a in ax[0]:
    a.set_xticklabels([])

for a in ax[1]:
    a.set_xlabel('$t$')
    a.set_ylim(0, 1.1)

#ax[1][1].set_yticklabels([])

plt.tight_layout(pad=0.05, w_pad=0.4, h_pad=0.2)

plt.savefig('fig/tunnel_applicability.eps')

# Different formulas

In [ ]:
def tunnel(E):
    return 4 / E * np.exp(- 2 / 3 / E)

def tunnel_ADK(E):
    return 2 * np.exp(2) / np.pi / E * np.exp(-2 / 3 / E)

@np.vectorize
def posthumus(E):
    if E < 1 / 16:
        return tunnel_ADK(E)
    else:
        return tunnel_ADK(1 / 16) + (1 - 1 / 16 / E) / 2 / np.pi

def field_squared(E):
    return 2.4 * E ** 2

def field_linear(E):
    return 0.8 * E

def tunnel_tong(E):
    return tunnel(E) * np.exp(-2 * 6 * E)

def tunnel_zhang(E):
    return tunnel(E) * np.exp((0.11714 * 16 ** 2 * E ** 2 - 0.90933 * 16 * E - 0.06034))

In [ ]:
plt.rc('xtick.major', size = 2)
plt.rc('ytick.major', size = 2)

fig, ax = plt.subplots()

plt.xscale('log')
plt.yscale('log')

power_min = -1.2
power_max = 1

E = np.logspace(power_min, power_max, 100)

styles = [[3, 1], [1, 1], [3, 1, 1, 1], [3, 1, 1, 1, 1, 1], [2, 1, 1, 1, 1, 1, 1, 1]]

plt.plot(E, tunnel(E), label='Tunnel')
plt.plot(E, posthumus(E), label='Posthumus', dashes=styles[0])
plt.plot(E, tunnel_tong(E), label='Tong', dashes=styles[1])
plt.plot(E, tunnel_zhang(E), label='Zhang', dashes=styles[2])
plt.plot(E, field_squared(E), label='Bauer', dashes=styles[3])
plt.plot(E, field_linear(E), 'C6-o', label='Motionless', ms=2, markevery=5)
plt.legend(fontsize=8)

plt.xlim(xmin=10 ** power_min, xmax=10 ** power_max)
plt.ylim(2e-3, 3)
plt.xlabel(r'$E$')
plt.ylabel(r'$w_\mathrm{i}$')

plt.tight_layout(pad=0.2)
plt.savefig('fig/ionization_models.eps')